## Preprocessing

In [7]:
#!pip install tensorflow

In [8]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd

# File path
file_path = "C:\\Users\\8saks\\Documents\\GitHub\\deep-learning-challenge\\charity_data.csv"

# Read the csv file
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
print(df.head())

        EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     

In [9]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = df.drop(columns=['EIN', 'NAME'])

In [10]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [11]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = df['APPLICATION_TYPE'].value_counts()

In [12]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
replace_applications = list(application_counts[application_counts < 500].index)

# Replace in dataframe
for application in replace_applications:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(application,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [13]:
# Look at CLASSIFICATION value counts for binning
classification_counts = df['CLASSIFICATION'].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [14]:
# Get CLASSIFICATION counts greater than 1
classification_counts_gt_1 = classification_counts[classification_counts > 1]

# Display these classifications
print(classification_counts_gt_1)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [16]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [17]:
# Convert categorical data to numeric with `pd.get_dummies`
df = pd.get_dummies(df)

In [19]:
# Split our preprocessed data into our features and target arrays
y = df["IS_SUCCESSFUL"].values
X = df.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [21]:
# Define the model - deep neural net
number_input_features = len(X_train[0])  # This assumes X_train is your feature data
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3520      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
import numpy as np

# Convert the data to float32
X_train = np.array(X_train).astype('float32')
y_train = np.array(y_train).astype('float32')

# Now try training the model again
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 6s 5ms/step - loss: 52219.2070 - accuracy: 0.4945
Epoch 2/100
804/804 [==============================] - 6s 7ms/step - loss: 18069.9121 - accuracy: 0.5129
Epoch 3/100
804/804 [==============================] - 4s 5ms/step - loss: 17480.5254 - accuracy: 0.4833
Epoch 4/100
804/804 [==============================] - 3s 4ms/step - loss: 29877.0156 - accuracy: 0.5074
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 19123.3281 - accuracy: 0.5101
Epoch 6/100
804/804 [==============================] - 4s 5ms/step - loss: 12956.5322 - accuracy: 0.5168
Epoch 7/100
804/804 [==============================] - 5s 6ms/step - loss: 10302.0693 - accuracy: 0.5152
Epoch 8/100
804/804 [==============================] - 3s 4ms/step - loss: 7943.3813 - accuracy: 0.5186
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 15379.7354 - accuracy: 0.5211
Epoch 10/100
804/804 [==============================] - 

804/804 [==============================] - 2s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 81/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6913 - accuracy: 0.5321
Epoch 82/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 83/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 84/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 85/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6911 - accuracy: 0.5321
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 0.5321
Epoch 89/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6912 - accuracy: 

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 1.0693 - accuracy: 0.6594 - 505ms/epoch - 2ms/step
Loss: 1.0692652463912964, Accuracy: 0.6593586206436157


In [27]:
# Export our model to HDF5 file
# Save the model
nn.save("AlphabetSoupCharity.h5")